# Introduction
Flaws in a computer system would allow the attacker to access the system. Flaws can be design flaws, hardware flaws(hardware trojans), flaws in software programs and human factor. Attacker needs just one of these flaws to gain access to your system. There are a quite a few software programming flaws at application and system software level that can be exploited. Of these 1. Buffer overflows and overreads 2. Heap: double free, use after free 3. Interger overflows and 4. Format strings are well known system software flaws.  
Approaches to build secure systems
* Design flawless systems : Not practicle to delvelop these systems with windows and linux in real world.
* Isolate systems : We build systems with flaws and then encapsulte these systems in sandbox environments, virtual machines and trusted execution environments.
* Detect and mitigate attacks : Anti-virus software would detect malicious code, based on certain characteristics during execution or based on certain characteristics of the binary of that particular executable.

## Program binaries
Executables(.elf(executable linker format) on linux, .exe on windows) are stored in hard disk. When you run the executable, the operating system creates a process, runs the excutable in the process on the RAM.

ELF format : It is a format for object files, libraries and executables.  

Object files start with the ELF header which describes the file organization, followed by different sections for code, data, symbols etc. At the end it has a section header table, which will help to locate the various sections.
 * ELF header : It has a identication, a magic number indicating that it is a .elf file. Type, weather the file is object file, shared obejct, executable or a core file. Machine type, for which processor this file is compiled for. Entry, virtual address where program begins execution(for executables). It also has a pointer to the section header table and the number of sections.
 * Section header table : This table has the section name, type of the section(PROGBITS, SYMTAB, NOBITS etc), virtual address where each section will be loaded(for executable), offset of the section within the file, size of the section and a few other columns.
 
Executables also have a similar structure. Executable files have segments instead of sections, it has program header table instead of section header table. The remaining is the same. Program header table helps to locate various file's segments. Again the segments could be for code, data, symbol table etc.
* Program header table : This table has type of the segment(loadable segment/shared lib/etc), offset of the segment in the ELF file, virtual address where the segment is to be loaded when exeuted, size of the segment and flags(read/write/execute) columns.

Process : When we run the executable from the shell, operating system calls fork() method which creates a new child process of the shell process. Child process will call execlp("hello", "", NULL) method to run the executable. The parent process(shell in this case) will call wait() method to wait for the child to complete execution and return.  
When the execlp system call is made, the OS would create a new virtual address base for the new process. Then it would load segments from elf file into this virtual address base segments. Code and intructions will be copied to the text segment of the virtual address base, global data members of the program will be copied into the data segment, dynamic data(new/malloc) will be allocated in the heap segment and local variables/function calls will be loaded into the stack segment of the virtual address base.

Example commands
* readelf -h hello.o : Command to get the ELF header details of hello.o object file.
* readelf -S hello.o : Command to get the ELF section header table.
* readelf -l hello : Command to get the ELF program header table of hello executable file.
* objdump --disassemble -all hello : Command to get assembly code from ELF files.
* ps -ae | grep hello : Get the pid of the running hello executable.
* sudo cat /proc/6757/maps : Display the virtual address base memory map of the process(6757 is pid).

## Buffer Overflows
Buffer overflows can be in the stack, in the heap or it can be a return-to-libc attack. If we try to access memory outside the allocated size then it is called a buffer overflow, for example with char buffer\[10\], if we try to access buffer\[10\] then it is a buffer overflow, this will return the value at location after buffer in the stack(which could be anything on the program stack, other variables, frame pointer, return address etc.). Using buffer overflow an attacker can modify return address to point to his attack code, which will run his code when the current function returns. Normally the attacker will fill the buffer with his attack code, and then use buffer overflow to modify current stack frame return address to point to his code. This is known as the buffer overflow attack. The attacker must fill the buffer with the machine code of his attack code, they use tools like obejctdump to get the machine code. For example the below code can be used for buffer overflow attack, as strcpy tries to copy till it see's a NULL('\0') character and the program is trying to copy a bigger array into a smaller one.
```
char large_string[128];
char buffer[48];              //defined on stack
...
strcpy(buffer, large_string);
```
The attacker fills up the start of the large_string with his attack code and the remaining bytes with the address to his attack code(buffer start address). In this way he can get the buffer to overflow into the stack and fill the return address with his attack code address.

Example commands
* (gdb)info registers eip esp ebp : Display the contents of the registers, instruction pointer, stack pointer, current stack frame base pointer.
* (gdb)x/32x $esp : Display the contents at the stack pointer, 32 words(memory locations) in hexa.
* (gdb)disassemble : Display the assembly code.
* (gdb)si 3 : Runs 3 assembly instructions(si runs a single assembly instructions).
* (gdb)info local : Display the contents of all locals variables.

## Buffer Overflow Example
In this example we demonstrate how to run the shell from the attack code, this gives the attacker access to the system via the shell.
```
int copier(char *str)
{
    char buffer[100];
    strcpy(buffer, str);
}

int main(int argc, char *argv[])
{
    copier(argv[1]);
    printf("Done!\n");
}
```
Here if arg\[1\] is greater than 100 bytes then a overflow will occur. If the overflow crosses the function stack frame then previous stack frame pointer and return address will be modified. If only previous stack frame pointer is modified and return address is not modified, program will return to main and continue to execute when exiting main it gives a segmentation fault.

## Preventing Buffer Overflows
There are two prevention techniques canaries and non executable stack.
* Canaries : It is a compiler defence mechanism. When ever a stack frame gets created(for a function) the compiler would insert a canary in the stack after the frame pointer by adding additional instructions. Canary is a random number, if this number is modified then the compiler would throw stack overflow error at the time of returning from the function(additional instructions inserted for this check), thus preventing buffer overflow attacks. For GCC we have to use -fstack-protector compiler option to enable canaries.
* Non executable stack : This is implemented in the hardware, processor. This is implemented with a W^X bit(write exclusive or execute), when this bit is enabled for a segment(for example say the stack segment) then one can either write to that segment or execute from that segment, we cannot do both. The stack segment W^X bit is set, thus preventing buffer overflow attacks. Both Intel and AMD processors support this bit. Very effective as it is done by hardware and uses only a single bit per segment, but there are legitimate cases to execute code from stack, like the java just-in-time runtime, which generates code at runtime in stack and runs it. This is not possible with this bit.

## Return to libc buffer overflow attack
In this attack code is not executed from the stack, this way the attack would run even if the W^X bit is set for that stack. libc is the standard c library. The return to libc attack also would overflow the buffer and modify the return address to some arbitary address. But this arbitary address points to some function in libc. As this code is present in code segment of the process(not stack), it executes. system is one such function that the attackers can use from libc. For example system("/bin/bash") will open the bash for the attacker, then the attacker can run any command from bash.    
So the attacker needs to find the address of the system function in libc, overflow the buffer with this address and fill up the return address with it. Then fill up the next location in stack with an address that points to the string /bin/bash(in virtual memory). We can get both these addresses with gdb from libc.  

Example commands
* (gdb)p system : Prints the address of the system function in libc.
* (gdb)info proc map : Displays the memory mappings of the current process, which also has the memory range of libc.
* (gdb)find 0xf7e08000,0xf7fb9000,"/bin/sh" : Find the string in the given address range and return its address.
* (gdb)x/s 0xf7f6102b : Dump the string at the given address.

## Return oriented programming attacks(ROP)
Return to libc attack is limited to only using functions from libc. ROP is not limited to using functions only from libc, it can execute any arbitary code. We have to build the instructions(code) that we want to execute. To build the attack code we have to find the gadgets in libc code by statistically analysing the libc code. Gadget is a small sequence of instructions followed by a return. Then find and stitch the gadgets to build the target payload. To stitch the gadgets we have to place the gadgets in the stack one top another in order required. With each gadget having a return statement, these gadgets execute one after the other. To find all the gadgets in libc we find all the return op codes, once we find the return op code we look at the previous instructions and try to form useful gadgets. 

## Address Space Layout Randomization(ASLR)
Address space layout randomization is a technique used to prevent attacks like return to libc and ROP. These attacks are highly dependent upon knowing where these functions reside in memory, assumes that many systems use the same address mapping, therefore one attack may spread easily. ASLR randomizes the address space layout of the process, each process execution would have a different memory map(randomize the location of libraries, heap, stack, etc for each run of the application), thus making it difficult for the attacker to run exploits. There are 2 well known techniques to implement ALSR, load time relocatable and position independent executable/programmable independent code.
* Load Time Relocatable(LTR) : Here the work is done by the loader, modifies the executable while loading, moves the code segments randomly. This means the compiled code can only have place holders for addresses, these palce holders will be filled in during load time using relocatable tables(present in compiled libraries and executable). Limitations are that it has slower load time, it needs writable code segment which could pose security problems and each program needs a customised copy of a shared library instead of using the same copy of the shared library.  
* Programmable Independent Code(PIC) : This tecnique gets rid of all the limitations in LTR. Here instead of relying on the loader to change all the addresses, it uses relative addressing and a special table called global offset table(GOT) is used. GOT has the location of the absolute address of varaibles and functions, assembly instructions are updated to use the addresses from the GOT. GOT is present in the data segment and constructed/filled just once during execution. Limitation is that the runtime overheads have increased.  
Attackers managed to bypass ASLR, techniques like brute force, timing attacks, return-to-plt and overwriting the GOT are used.

Example commands
* cat /proc/sys/kernel/randomize_va_space : Check if ALSR is enabled in the linux kernel, a value 2 means a highest level of randomization.
* cat /proc/14621/maps : Display the memory map of the process with the process id 14621.
* readelf -r libmylib.so : Get the relocatable table for libmylib.so library.
* (gdb) disassemble set_mylib_int : Disassemble the assembly code of the function set_mylib_int.

## Buffer Overreads
If a program in some way allows to read data past the buffer size then it is known as buffer overread. If the data after the buffer is some sensitive data, then the attacker can be hold of that sensitive data. Below is a simple example of a buffer overread.
```
char some_data[] = "some data";
char secret_data[] = "TOPSECRET";

void main(int argc, char **argv)
{
    int i = 0;
    int len = atoi(argv[1]); //length to be printed
    
    while(i < len)
    {
        printf("%c", some_data[i]);
        i++;
    }
    
    printf("\n");
}
```
Here some_data array has been initialized to some specific size but the user can read more data than what is in some_data array(from adjacent memory) by inputting a higher length to print(program's first argument).In this example "TOPSECRET" will be present in the adjacent memory hence it is a vulnerability. One real world example of buffer overread was the heartbleed attack which tagetted OpenSSL to steel information, OpenSSL did not check for the maximum length of the heartbeat message(payload length is present in the message), which ment the attacker could read past the buffer.

## Format String Vulnerabilities
Format string is used quite often by functions like printf, scanf and C++ stream objects, this could lead to a vulnerability. Normally the format string will have the format specifiers(like %d(int), %x(hexa decimal), %s(string)), format specifiers will be replaced with the arguments when forming the resulting string. One of the vulnerability of format string is insufficient arguments, example printf("%d %d %d\n", a, b), 3 format specifiers but only 2 arguments. In this case the value in location above b in the stack is printed for the 3rd format specifier. printf("%s%s%s%s%s%s"), may crash as this would try to get strings from arbitary locations present in the satck as there are no arguments to printf. The below code has the vulnerability to print contents of any memory location of the program.
```
static char s[1024] = "THIS IS A TOP SECRET MESSAGE!!!";
void main()
{
    char user_string[100];
    strcpy(user_string, "\xc0 \x96 \x04 \x08 %x %x %x %x %x %x %s");
    printf(user_string);
}
```
The purpose of this attack is to print the contents of s. In a real world example the user_string could be taken from the user or recieved on the network. '\xc0 \x96 \x04 \x08' is the address for the string s. printf works using 2 pointers, printer p points to the format specifiers in the stack and pointer ap which points to the arguments in the stack. %x's are added to the format string so that ap points to '\xc0 \x96 \x04 \x08' in the stack when %s format is being resolved by printf, this way the attacker can get the secret message.  
printf also has format specifiers that can change a value at a memory location, not just read. For example printf("12345%n", &i), the %n format specifier here would return the number of characters printed so far and load it to i. Using the same technique above to read any memory location, we can use the %n format specifier to write to any desired memory location with the number of bytes that printf has printed so far.

## Integer Vulnerabilities
There are 3 kinds of interger overflow vulnerabilites, width overflow, arithmatic overflow and sign/unsigned problems.
* Width overflow : Occurs when code tries to store a value in a variable that is too small to handle it.  

```
int main(int argc, char *argv[])
{
    unsigned short s;
    int i;
    char buff[80];
    
    //We get the length to copy from the first argument of the program
    i = atoi(argv[1]);
    //Since s is short type if i is greater than 65535 a width overflow occurs and the value in s is truncated
    //For example if i is 65536 then s would be 0
    s = i;
    //Will not return since s is 0
    if(s >= 80)  //Local buffer is of 80 size
    {
        return -1;
    }
    
    //For memcpy the programmer uses i, which is 65536 in this case, which results in a buffer overflow
    //This buffer overflow can be exploited as we have seen before
    memcpy(buf, argv[2], i);
}
```

* Arithmetic overflow : When we do arithmethic operations the result can overflow the maximum value the variable can fold, it that case the result will be truncated.

```
int my_function(int* array, int len)
{
    int *my_array, i;
    
    //There could be a arithmetic overflow here
    //If the user passes in a very high len, len * sizeof(int) could overflow int
    //Thus the value passed to malloc could be very small due to overflow and my_array is a very small memroy size
    my_array = malloc(len * sizeof(int));
    if(my_array == NULL)
    {
        return -1;
    }
    
    //But the for loop runs on len value, which is a very high value
    //This would result in my_array being overflowed
    //This buffer overflow can be exploited as we have seen before
    for(int i = 0; i < len; i++)
    {
        my_array[i] = array[i];
    }
    
    return my_array;
}
```

* Signed/Unsigned Vulnerabilities : Sign interpretation is done using the most significant bit, this can lead to unexpected results during comparisions and arithmetic.

```
//len is defined as int which is signed
//On the other hand, the 3rd argumnet to memcopy(number of bytes to copy)  is an unsigned integer
int copy_something(char* buf, int len)
{
    char kbuf[800];
    
    //If user gives len of -1, function will not return since -1 is not greater than 800
    if(len > sizeof(kbuf))
    {
        return -1;
    }
    
    //When memcopy executes, there is a implicit type cast for the 3rd parameter from signed int to unsigned int
    //This cast will make the len a very high value, resulting in a buffer overflow during copy 
    return memcopy(kbuf, buf, len);
}

```

Stagefright is a solfware library implemented in C++ for android. An attack was deviced using integer vulnerabilities in this library, it affected thousands of android devices.

## Heap exploits
Heap is a pool of memory present in the processes address space where dynamically alocated memory resides. Stack is fast, managed by compiler and used for temporary data, on the other hand heap is slow, manually managed by user and used for persistent data(across function calls). There are several different types of implementation/algorithms for heap management with the functions malloc/new and free/delete. These algorithms have to do a trade of between speed of memory management and memory fragmentation. Some of the famous implementations are ptmalloc2/ptmalloc3 from glibc, tcmalloc from google and jemalloc from android. Internally malloc/new uses brk and mmap syscalls to obtain memory from the OS for that process, a program will start with a heap size of 0, memory is allocated dynamically as new calls to malloc/new are made. The heap memory is structured as arena's, heaps and chunks. Arena is the full area, one arena per thread, each arena can have multiple heaps. Heap is the amount of memory allocated into arena by a single call to brk(typically 132 KB) when there is not enough memory in the arena. Heap is split into chunks of different sizes depending on the programs malloc and free calls. Free calls will not remove heaps from the arena, they remain there to be used by the next malloc/ new calls. The arenas are linked on a linked list, heaps in a arena are also linked on a linked list, similarly chunks in a heap are linked on a linked list, hence heap is not a continuous memory. Chunks are of 2 types, free chunks and allocated chunks, on calling free/delete allocated chunk is converted to free chunk,free chunks are linked in a doubly linked list to easily find the right chunk for the next malloc/new. To improve performance there are multiple doubly linked list's for free chunks, each list for particular size range of the chunks called bins. During free, if the previous or the next chunck is free, it can coalesce these adjacent chunks and form a much larger free chunk, this way fragmentation is reduced.  

Double free : Most double frees are not detected. The second free would add a second duplicate chunk into the free double linked list. If malloc after this uses one of these 2 chunks, then you would have the same chunk allocated and in the free linked list. This would allow to change the forward and backward pointers of the chunk in the free double linked list using the allocated chunk. By appropriately setting the forward and backward pointers we can force the attack code to execute.

```
char payload[] = "\x33\x56\x78\x12\xac\xb4\x67";

void fun1()
{
}

void main()
{
    char *a = malloc(10);
    char *b = malloc(10);
    char* c;
    
    fun1();
    free(a);
    free(b);
    free(a);
    
    c = malloc(10);
    *(c + 0) = GOT entry-12 for fun1;
    *(c + 4) = payload;
    some malloc(10);
    fun1();
}
```
In the above example as discussed above, after double free of a and malloc of c, c and the second a in free linked list will point to the same memory chunk. Then the attaker tries fills c and (c + 4) with GOT and payload with scanf or through a network packet. This will break the free linked list with a's forword pointer as GOT entry and backward pointer as payload. On the next malloc due to free linked list link changes, payload will be written into the GOT entry, thus when fun1() is called again payload gets executed.  
The double free is one of the heap exploits, there are other heap exploits like heap overflows, heap spray, use after free and mata data exploit. All these exploits follow the same principle that use the management of free and allocated chunks to device the attacks as shown here.

Example commands
* sudo cat /proc/6757/maps : Display the virtual address base memory map of the process(6757 is pid), memory map will also show the various arenas.

## Access Control
Any system that is designed to be secure should have some access control policies for confidentiality and integrity. Access control defines who can access what. Who are the subjects(user/process/application), what are the objects(files/programs/sockets/hardware), access is the various operations(read/write/execute/share) subjects can perform on objects. Access control can be at various levels application, middleware, OS and hardware.
* Hardware access control : It should have policies to protect OS from applications, ensure that one application cannot interfere with another application, prevent one application hogging the entire system. To achieve these 3 policies hardware has the mechanisms paging unit, privilege rings and interrupts. Paging unit isolates one application address space from another application address space, thus protecting one application from another. Privilege rings protect OS from applications. OS runs in ring 0 and applications run in ring 3. Applications running in ring 3 cannot access, read or write anything running in ring 0. Interrupts are used to prevent one application hogging the system, interrupts are used by the schedular to context switch thus preventing from one application running.  
* OS access control : It should have policies so that only authenticated users can use the system, one user's files should be protected from other user's and fair allocation of resources without starvation. OS uses the machanisms user authentication, access controll for files/objects and scheduling to achieve these policies. OS access control is based on user(indentity) and access rules which state what requesters are allowed to do. These rules are set by the administrator.
* Linux access control : Subjects are users, groups, root user and processes(gets the permissions and privileges of the user who creates it). Objects are files, directories, sockets, process, process memory, file descriptors and so on. All users and their passwords are stored in /etc/shadow, /etc/passwd has information about the user like uid, gid, home directory of that user etc. When the process starts setuid(user ID) and setgid(group ID) are used to set the user and group id to that process, then the process will have the permissions and privileges of that user and group. On login a shell process is executed with the logged in user, any child process started from this shell will execute with the same user and group. sudo/su can be used to elevate the privileges to the root user. Every object in linux has read, write and execute(R, W and X) permissions. There are also 3 different types of users associated with every object onwer, group(to which the onwer belogs to) and others. For each of these 3 users the object specifies R, W and X permissions.  
The entire linux security checks are done when the user tries to the get the file descriptors(access control permissions are checked by the operating system), once the user/process has the file descriptor no other checks are done. There are 2 ways to get a file descriptor, open a file or get it from another process(from parent or through shared memory/sockets). This way a root user can open a file and send the desciptor to a normal user, so the normal user can read/write into that file without having permissions for that file.  
Process is both subject and object in linux. When debugging ptrace system call will allow one process to observe and control the other(GDB will control the process you are debugging).  
This form of access control based on users has some drawbacks, like we cannot control information flow and the root user can do anything. To address these problems a new access control based on information flow policies was introduced. Mandatory access control is a information flow policy access control, it has to be enabled to use it, default is the access control based on users.

Example linux commands  
* id : Display all the ids related to the current user.  
* passwd : Command used by the user to change his or her password.  
* chown : Change the ownership of a file.  
* chmod : Change the permissions of a file.  
* chgrp : Change the group of a file.  

## Confinement in applications
How do you run an entire untrusted application(from the internet) or part of a application that is untrusted(module or library) in your systems. Possible solutions are to use air gapped systems(isolated system not connected to the network), virtual machines or containers. All these solutions require a seperate system, virtual machine or container to be setup.  
One way to acheive confinement is to break the large application(run in a single process) into small modules. Run each of these module as a different process and use RPC to communicate between the modules. Therefore if a vulnerabilbity is exploited by a attacker in any one of these process modules, the attacker would gain access to only that particular module, thus we are able to contain the amount of damage. These modules should be designed with the principle of least privileges, each module is given minimum privileges only enough to carry out its job, narrow interfaces between subsystems that only allow necessary operations, and security management should be done outside the application(example by the OS).  
The disadvatage with this architecture is that it is slow as RPC is slow and a lot of context switching is required for the different processes to communicate.

Example linux commands  
* chroot : define the root file system a process can see, we can give a seperate file system for that process.
* setuid : set the uid of a process to confine what it can do.

## Software fault isolation
In this architecture all the modules are run in a same process, but each module is run within a confined environment, thus removing performance overheads. Here process space is partitioned into logical domains, each domian has its own code area, data area and a unique id, code in one domain is not allowed to read/modify data in another domain, code in one domain cannot call code in another domain, it is restricted to its code and data. The only way one doamin can communicate with another domain is through a low cost cross-domain RPC interface not involving OS(no context switching).  
To acheive this the process user space is divided into several different memory segments, the segments are aligned in a way such that within each segment the higher order bits of the memory addresses are the same, this higher order bits value will be the unique ID of that segment. When calling code or data from a segment the higher order bits are checked, if not same the call is not allowed. 
To achieve segmentation, at the time of loading a object file into a segment we pass through the entire object file and check that every branch/load/store instruction is into the same segment. If the check passes the object is loaded into the segment. For instructions where adresses cannot be resolved during load time, check is done during runtime.  
The cross-domain RPC is implemented using jump table(present in the domain), call stub(seperate domain), return stub(seperate domain). For every legal RPC call there is a look up in the jump table, this invokes the call stub, then the call stub invokes the method in the second domain, the return would be passed through the return stub. There is also a seperate domain for system calls, so any system call would go through the stub to this system domian, which checks the permissions of the domian and performs the call.